<a href="https://colab.research.google.com/github/RiseAboveAll/PYTORCH_Learning/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequential Data

- Example of sequential data : Text , Speech, Financial Data

<h1> Types Of Sequence Data : </h1>

1. Time Series Data

  - Any continues value measurement recorded periodically 

2. Text 

  - Can be treated as sequence 

  - In ML we used to create Bag of Words, Using this we used to loose information.

<h1> What is Sequence ? </h1>

Length of data is measured by time , represented by letter 'T'.

Shape of Data = N x T x D

N -> N.O of Samples

T -> Time Step

D -> N.O of Features

Most of the time we have variable length sequences, i.e if T is variable i.e not constant length of sequence in data will differ but we require constant length of all . Example sentences , words in sentences differ. We can pad the sequences with 0s so that all the sequence at T(n) become constant length. 

In pytorch , instead of having all the sequences padded to the same length as the longest sequence, just do this accross single batches. Pytorch try to group sequences on the basis of similarity in length , hence batches will be random but not as random as it would be.  



# Forecasting

- We have time series , we have to predict next values of the time series.

- Number of future steps we want to predict is called Horizon


# RNN

**In ANN we used to calculate the output of hidden node using the input vector and output of output node using the output of hidden node as input . X -> H -> Y_hat.**

**H = a(W.T . X + b)**

**Y_hat = a(W.T . H + b)**

Whereas,

**In RNN, the *Hidden Node*  loop back to itself, i.e in other words the *Hidden Node* does not only depends on the input, but also on it's own previous (time-stamp) value. This *Hidden Node is also called *Hidden State*. This Hidden State is non linear function of past values. This loop-back to previous value states that there is time delay of one step**

<h3> RNN Equation </h3>

Ht = a(Wxh.T . X + Whh.T . Ht-1 + b)

Y_hat = a(W.T . Ht + b)

<h3> Calculation </h3>

Given : x1,x2,x3,....,xt

Shape(Xt) : D

First Step : 

-> h1 = a(Wxh.T . x1 + Whh.T . h0 + b)

-> y_hat1 = a(W.T . h1 + b)

Now we have x2 & h1 :

-> h2 = a(Wxh.T . x2 + Whh.T . h1 + b)

-> y_hat2 = a(W.T . h2 + b)

We can repeat this process till we calculate hT and y_hatT. 

**Each y_hatT depends on x1,x2,x3,...,xt, we get yhat for each corresponding x1,x2,x3,....,xt...xT. xT gives us the final prediction , i.e final y_hat , hence all the previous time-stamps y_hat will be removed. There are instances when we do not discard the previous y_hats , instances like neural machine translation. In machine translation , both input and output are senteces but in different languages. Both the input and output are sequences, capture predctions at each time point. 

<h3>Classification Probability</h3>

- In ANN and CNN after applying softmax the output prediction is probability distribution. P(y=k|X)

- For neural machine translation, it is also a classification, but we will have probability of yt=k given something. P(y=k|?) . 

What is this 'Something'?

![RNN](https://raw.githubusercontent.com/RiseAboveAll/PYTORCH_Learning/master/RNN.PNG)

We can say that y_hatT depends on every X in the input sequence. Hence:

P(y1=k|x1)

P(y2=k|X2,X1)

P(y3=k|X3,X2,X1)

.
.
.
.

P(yT=k|XT,...,X3,X2,X1)

RNN accounts for all the previous words in the sentence not only the previous words. 

